In [124]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [130]:
df = pd.read_csv('google_books.csv')
print(df.shape)
df.head()

(1116, 14)


,title,subtitle,authors,publisher,month,year,categories,pageCount,listPrice,images,isEbook,ratingsCount,averageRating,description
0,The Litigators,NaN,John Grisham,Dell,10,2011,Fiction,496,9.99,False,True,137,3.5,The partners at Finley & Figg often refer to t...
1,Ragtime,A Novel,E.L. Doctorow,Random House,11,2010,Fiction,336,14.99,True,True,57,3.5,Selected by the Modern Library as one of the 1...
2,Her Perfect Man,NaN,Mary Lynn Baxter,Silhouette,5,2011,Fiction,192,3.99,False,True,1,5.0,One look at firebrand Katherine Mays and Bryce...
3,The Vanished Man,A Lincoln Rhyme Novel,Jeffery Deaver,Simon and Schuster,8,2012,Fiction,577,NaN,False,False,19,4.0,Hunting down a killer and master illusionist w...
4,One Sunday,A Novel,Carrie Gerlach Cecil,Simon and Schuster,2,2013,Fiction,255,NaN,False,False,6,4.0,"When her unborn child's health is put at risk,..."


In [131]:
df.dropna(subset=['description'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape
df

,title,subtitle,authors,publisher,month,year,categories,pageCount,listPrice,images,isEbook,ratingsCount,averageRating,description
0,The Litigators,NaN,John Grisham,Dell,10,2011,Fiction,496,9.99,False,True,137,3.5,The partners at Finley & Figg often refer to t...
1,Ragtime,A Novel,E.L. Doctorow,Random House,11,2010,Fiction,336,14.99,True,True,57,3.5,Selected by the Modern Library as one of the 1...
2,Her Perfect Man,NaN,Mary Lynn Baxter,Silhouette,5,2011,Fiction,192,3.99,False,True,1,5.0,One look at firebrand Katherine Mays and Bryce...
3,The Vanished Man,A Lincoln Rhyme Novel,Jeffery Deaver,Simon and Schuster,8,2012,Fiction,577,NaN,False,False,19,4.0,Hunting down a killer and master illusionist w...
4,One Sunday,A Novel,Carrie Gerlach Cecil,Simon and Schuster,2,2013,Fiction,255,NaN,False,False,6,4.0,"When her unborn child's health is put at risk,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,Abstract Algebra: An Introduction,NaN,Thomas W. Hungerford,Cengage Learning,7,2012,Mathematics,616,69.49,True,True,2,3.0,Abstract Algebra: An Introduction is set apart...
1107,Introduction to Geophysical Fluid Dynamics,Physical and Numerical Aspects,"Benoit Cushman-Roisin, Jean-Marie Beckers",Academic Press,1,2011,Mathematics,828,NaN,True,False,4,5.0,Inhaltsverzeichnis: I Fundamentals - 1: Introd...
1108,Cambridge Primary Mathematics Stage 5 Learner'...,NaN,"Emma Low, Mary Wood",Cambridge University Press,5,2014,Mathematics,129,NaN,True,False,1,5.0,Written for use with the Cambridge Primary Mat...
1109,Mathematical Statistics with Applications,NaN,"Dennis Wackerly, William Mendenhall, Richard L...",Cengage Learning,10,2014,Mathematics,944,NaN,False,False,2,3.5,In their bestselling MATHEMATICAL STATISTICS W...


In [132]:
for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    print(words)

['Selected', 'by', 'the', 'Modern', 'Library', 'as', 'one', 'of', 'the', '100', 'best', 'novels', 'of', 'all', 'time', 'Published', 'in', '1975', ',', 'Ragtime', 'changed', 'our', 'very', 'concept', 'of', 'what', 'a', 'novel', 'could', 'be', '.']
['An', 'extraordinary', 'tapestry', ',', 'Ragtime', 'captures', 'the', 'spirit', 'of', 'America', 'in', 'the', 'era', 'between', 'the', 'turn', 'of', 'the', 'century', 'and', 'the', 'First', 'World', 'War', '.']
['The', 'story', 'opens', 'in', '1906', 'in', 'New', 'Rochelle', ',', 'New', 'York', ',', 'at', 'the', 'home', 'of', 'an', 'affluent', 'American', 'family', '.']
['One', 'lazy', 'Sunday', 'afternoon', ',', 'the', 'famous', 'escape', 'artist', 'Harry', 'Houdini', 'swerves', 'his', 'car', 'into', 'a', 'telephone', 'pole', 'outside', 'their', 'house', '.']
['And', 'almost', 'magically', ',', 'the', 'line', 'between', 'fantasy', 'and', 'historical', 'fact', ',', 'between', 'real', 'and', 'imaginary', 'characters', ',', 'disappears', '.']
[

In [133]:
sentence

"Henry Ford, Emma Goldman, J. P. Morgan, Evelyn Nesbit, Sigmund Freud, and Emiliano Zapata slip in and out of the tale, crossing paths with Doctorow's imagined family and other fictional characters, including an immigrant peddler and a ragtime musician from Harlem whose insistence on a point of justice drives him to revolutionary violence."

In [134]:
pattern = r"[^\w]"
print(re.sub(pattern, " ", sentence))

Henry Ford  Emma Goldman  J  P  Morgan  Evelyn Nesbit  Sigmund Freud  and Emiliano Zapata slip in and out of the tale  crossing paths with Doctorow s imagined family and other fictional characters  including an immigrant peddler and a ragtime musician from Harlem whose insistence on a point of justice drives him to revolutionary violence 


In [135]:
count_vectorizer = CountVectorizer()

In [136]:
bag_of_words = count_vectorizer.fit_transform(df['description'].dropna())

In [137]:
feature_names = count_vectorizer.get_feature_names()

In [138]:
bag = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

In [139]:
bag

,000,000000,0001,007,0245,05,08,0px,10,100,...,वम,वस,वह,शन,शब,षय,सकत,सत,सर,सहज
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1107,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1108,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
bag.columns

Index(['000', '000000', '0001', '007', '0245', '05', '08', '0px', '10', '100',
       ...
       'वम', 'वस', 'वह', 'शन', 'शब', 'षय', 'सकत', 'सत', 'सर', 'सहज'],
      dtype='object', length=21536)

In [141]:
## use tf-idf to weight specific words based on their occurance in the series
## as opposed to doing a simple count 

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
values = tfidf_vectorizer.fit_transform(df['description'].dropna())
feature_names = tfidf_vectorizer.get_feature_names()
tfidf = pd.DataFrame(values.toarray(), columns = feature_names)

In [ ]:
## i am taking the mean tf-idf value for each feature/word in the dataframe
## after that, sorting them them in descending order so highest mean values are first
## then, i am taking the 400 features/words with the highest tf-idf mean-value
## then i am one-hot-encoding these 400 features and putting them back in the dataframe

In [172]:
words_scores = tfidf.apply(np.mean, axis=0)

In [173]:
words_scores

000       0.003366
000000    0.000211
0001      0.000108
007       0.000079
0245      0.000108
            ...   
षय        0.000084
सकत       0.000084
सत        0.000084
सर        0.000084
सहज       0.000084
Length: 21247, dtype: float64

In [143]:
words_idx = words_scores.sort_values(ascending=False)[0:400].index

In [144]:
filtered = tfidf[words_idx]
one_hot_words = filtered.applymap(lambda x: 1 if x > 0 else 0)

In [145]:
one_hot_words.apply(np.sum, axis = 0)

new           459
book          433
life          352
world         393
data           78
             ... 
greatest       36
words          36
gives          43
university     27
wide           40
Length: 400, dtype: int64

In [146]:
one_hot_words

,new,book,life,world,data,war,history,story,author,edition,...,cookbook,born,voice,music,behavior,greatest,words,gives,university,wide
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1107,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1108,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1109,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
pd.concat([df, one_hot_words], axis = 1)

,title,subtitle,authors,publisher,month,year,categories,pageCount,listPrice,images,...,cookbook,born,voice,music,behavior,greatest,words,gives,university,wide
0,The Litigators,NaN,John Grisham,Dell,10,2011,Fiction,496,9.99,False,...,0,0,0,0,0,0,0,0,0,0
1,Ragtime,A Novel,E.L. Doctorow,Random House,11,2010,Fiction,336,14.99,True,...,0,0,0,0,0,0,0,0,0,0
2,Her Perfect Man,NaN,Mary Lynn Baxter,Silhouette,5,2011,Fiction,192,3.99,False,...,0,0,0,0,0,0,0,0,0,0
3,The Vanished Man,A Lincoln Rhyme Novel,Jeffery Deaver,Simon and Schuster,8,2012,Fiction,577,NaN,False,...,0,0,0,1,0,0,0,0,0,0
4,One Sunday,A Novel,Carrie Gerlach Cecil,Simon and Schuster,2,2013,Fiction,255,NaN,False,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,Abstract Algebra: An Introduction,NaN,Thomas W. Hungerford,Cengage Learning,7,2012,Mathematics,616,69.49,True,...,0,0,0,0,0,0,0,0,0,0
1107,Introduction to Geophysical Fluid Dynamics,Physical and Numerical Aspects,"Benoit Cushman-Roisin, Jean-Marie Beckers",Academic Press,1,2011,Mathematics,828,NaN,True,...,0,0,0,0,0,0,0,0,0,0
1108,Cambridge Primary Mathematics Stage 5 Learner'...,NaN,"Emma Low, Mary Wood",Cambridge University Press,5,2014,Mathematics,129,NaN,True,...,0,0,0,0,0,0,0,0,1,1
1109,Mathematical Statistics with Applications,NaN,"Dennis Wackerly, William Mendenhall, Richard L...",Cengage Learning,10,2014,Mathematics,944,NaN,False,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
## maybe try bi-grams/n-grams?
## use ngram_rangetuple(min_n, max_n) in sklearn
## default=(1, 1)

In [168]:
df.authors.value_counts()[:40]

Paulo Coelho            4
Jojo Moyes              4
Bill Bryson             4
William Shakespeare     3
J.K. Rowling            3
Francis D. K. Ching     3
Kristin Hannah          2
Stephanie Burgis        2
Sarah Rose              2
Tristan Gooley          2
Mary Oliver             2
Claudia Rankine         2
John O'Loughlin         2
Anna Todd               2
Margaret Atwood         2
Anthony Horowitz        2
William H. McRaven      2
Nigella Lawson          2
Aurélien Géron          2
Joseph Prince           2
Norman Stone            2
Khaled Hosseini         2
David Mamet             2
Lisa Wingate            2
Zed A. Shaw             2
T. S. Eliot             2
Stephen Fry             2
Philip Levine           2
Elif Shafak             2
Nora Roberts            2
James L. Stokesbury     2
Jonathan Safran Foer    2
Ina Garten              2
Randall Munroe          2
Thomas Keller           2
Stephen E. Ambrose      2
Liane Moriarty          2
Sam Harris              2
Stephen King